In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from statsmodels.sandbox.regression.gmm import IV2SLS 
from statsmodels.sandbox.regression.gmm import GMM
import statsmodels.api as sm
import matplotlib as mp

In [19]:
df = pd.read_csv('/Users/youziya/OneDrive - York University/MBAN 5110 Predictive Modelling/Midterm/midterm_partone.csv')

In [20]:
df.head()

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446


In [21]:
#run the ols regression
model_iv = sm.OLS(df["Inventory Turnover"],df[["Constant","Current Ratio","Quick Ratio",\
                                                                 "Debt Asset Ratio"]]).fit()
#making predictions, note that only the independent variables are in predictions 
endog_predict = model_iv.predict(df[["Constant","Current Ratio","Quick Ratio","Debt Asset Ratio"]])
#adding predictions to data table
df["Endogenous Param"] = endog_predict

In [22]:
model_2sls = sm.OLS(df["Stock Change"], df[["Constant","Endogenous Param",\
                                                              "Operating Profit","Interaction Effect",\
                                                             ]]).fit()
model_2sls.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Stock Change   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     8.530
Date:                Sat, 11 Nov 2023   Prob (F-statistic):           1.27e-05
Time:                        18:19:16   Log-Likelihood:                -1186.5
No. Observations:                1696   AIC:                             2381.
Df Residuals:                    1692   BIC:                             2403.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Constant              -0.0176      0.020     -0.896      0.370      -0.056       0.021
Endogenous Param       0.0011      0.001      1.827      0.068   -7.76e-05       0.002
Operating Profit      -0.1201      0.028     -4.319      0.000      -0.175      -0.066
Interaction Effect     0.0014      0.000      3.621      0.000       0.001       0.002
==============================================================================
Omnibus:                      368.832   Durbin-Watson:                   2.243
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3433.920
Skew:                           0.742   Prob(JB):                         0.00
Kurtosis:                       9.811   Cond. No.                         109.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### GMM

In [23]:
y_vals  = np.array(df["Stock Change"])
x_vals  = np.array(df[["Inventory Turnover","Operating Profit","Interaction Effect"]])
iv_vals = np.array(df[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])

In [24]:
class gmm(GMM):
    def momcond(self, params):
        # Now includes delta as the last parameter
        p0, p1, p2, p3, delta = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument

        # Calculate errors
        errors = endog - p0 - p1 * exog[:, 0] - p2 * exog[:, 1] - p3 * exog[:, 2]
        
        # Adjust moment conditions for delta
        g = np.column_stack((
            errors - delta,
            errors * exog[:, 0] - delta,
            errors * exog[:, 1] - delta,
            errors * inst[:, 0] - delta,
            errors * inst[:, 1] - delta,
            errors * inst[:, 2] - delta
        ))
        return g

# Update the initial values for the parameters, including delta
beta0 = np.array([0.1, 0.1, 0.1, 0.1, 0.1])  # Added an initial guess for delta

# Fit the model
res = gmm(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=6, k_params=5).fit(beta0)

res.summary()

Optimization terminated successfully.
         Current function value: 0.000007
         Iterations: 9
         Function evaluations: 17
         Gradient evaluations: 17
Optimization terminated successfully.
         Current function value: 0.000167
         Iterations: 7
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.000142
         Iterations: 19
         Function evaluations: 22
         Gradient evaluations: 22
Optimization terminated successfully.
         Current function value: 0.000143
         Iterations: 6
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.000143
         Iterations: 2
         Function evaluations: 4
         Gradient evaluations: 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.2426
Model:                            gmm   Prob (Hansen J):                 0.622
Method:                           GMM                                         
Date:                Sat, 11 Nov 2023                                         
Time:                        18:19:16                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.2038      0.193     -1.056      0.291      -0.582       0.174
p 1            0.0046      0.004      1.091      0.275      -0.004       0.013
p 2            0.6549      0.769      0.852      0.394      -0.852       2.161
p 3           -0.0143      0.017     -0.844      0.399      -0.047       0.019
p 4            0.0071      0.005      1.423      0.155      -0.003       0.017
==============================================================================
"""

In [28]:
class gmm(GMM):
    def momcond(self, params):
        # Include delta as the last parameter
        p0, p1, p2, p3, delta = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        # Calculate errors
        errors = endog - p0 - p1 * exog[:, 0] - p2 * exog[:, 1] - p3 * exog[:, 2]

        # Adjust moment conditions with the delta term
        g = np.column_stack((
            errors - delta,
            (errors - delta) * exog[:, 0],
            (errors - delta) * exog[:, 1],
            (errors - delta) * exog[:, 2],
            (errors - delta) * inst[:, 0],
            (errors - delta) * inst[:, 1],
            (errors - delta) * inst[:, 2]
        ))
        return g

# Update the initial values for the parameters, including delta
beta0 = np.array([0.1, 0.1, 0.1, 0.1, 0.1])  # Added an initial guess for delta

# Fit the model
res = gmm(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=7, k_params=5).fit(beta0)

res.summary()


Optimization terminated successfully.
         Current function value: 0.000101
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.001766
         Iterations: 8
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.001744
         Iterations: 6
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.001744
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                        2.958
Model:                            gmm   Prob (Hansen J):                 0.228
Method:                           GMM                                         
Date:                Sat, 11 Nov 2023                                         
Time:                        18:21:11                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0006   8.15e+05  -7.02e-10      1.000    -1.6e+06     1.6e+06
p 1            0.0004      0.000      1.024      0.306      -0.000       0.001
p 2           -0.1188      0.031     -3.861      0.000      -0.179      -0.059
p 3            0.0014      0.000      3.645      0.000       0.001       0.002
p 4           -0.0006   8.15e+05  -7.02e-10      1.000    -1.6e+06     1.6e+06
==============================================================================
"""

In [29]:
class gmm_with_delta(GMM):
    def momcond(self, params):
        # Unpack parameters, including delta as the last parameter
        p0, p1, p2, p3, delta = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument

        # Adjusted errors to include delta term
        errors = endog - (p0 + p1 * exog[:, 0] + p2 * exog[:, 1] + p3 * exog[:, 2]) - delta

        # Adjusted moment conditions with delta term
        g = np.column_stack((
            errors,
            errors * exog[:, 0],
            errors * exog[:, 1],
            errors * exog[:, 2],
            errors * inst[:, 0],
            errors * inst[:, 1],
            errors * inst[:, 2]
        ))
        return g

# Initial guesses for the parameters, including delta
# 4 original parameters + 1 delta
initial_params_with_delta = np.array([0.1, 0.1, 0.1, 0.1, 0.1])

# Create and fit the GMM model with delta term
model_with_delta = gmm_with_delta(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=7, k_params=5)
results_with_delta = model_with_delta.fit(initial_params_with_delta)

# Display the summary of the results
results_with_delta.summary()


Optimization terminated successfully.
         Current function value: 0.000101
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.001766
         Iterations: 8
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.001744
         Iterations: 6
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.001744
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3


LinAlgError: Singular matrix

Hansen J Statistic and its Probability:

The Hansen J statistic value is 0.2426, and the probability (p-value) associated with it is 0.622.
This high p-value (greater than the conventional threshold of 0.05) suggests that we cannot reject the null hypothesis of the validity of the instrumental variables. In simpler terms, the instruments appear to be appropriate for the model.
Coefficient Estimates:

The coefficients (p0, p1, p2, p3, p4) have been estimated with their respective standard errors and z-scores.
None of the coefficients appear to be statistically significant at conventional levels (e.g., 0.05) since all the p-values associated with them are greater than this threshold.
Interpretation of the Coefficients:

The coefficients represent the estimated impact of each variable (and potentially the bias 
�
δ) on the dependent variable 
�
y.
The lack of statistical significance suggests that the data does not provide strong evidence to confirm the relationships modeled between these variables and the dependent variable.
Industry Expert's Claim:

The expert's claim was that there is a bias in the moment conditions, represented as 
�
δ in the equation 
�
�
(
�
−
�
�
)
=
�
[
1
,
1
,
1
]
Z 
T
 (Y−XB)=δ[1,1,1], where 
�
δ has a non-zero value.
The fact that the Hansen J test does not reject the null hypothesis and that the coefficients, including the term for 
�
δ, are not statistically significant, suggests that the data does not strongly support the expert's claim of a bias.
Considerations:

The results should be interpreted with caution, as the lack of statistical significance could also be due to other factors such as insufficient sample size, poor model specification, or weak instruments.
It's also important to consider the context and theoretical foundation of the model. If the expert's claim about the bias has a strong theoretical basis, it might still be worth considering despite the statistical results.
In summary, based on the GMM results, the data does not provide strong evidence to support the industry expert's claim of a bias in the moment conditions. However, careful consideration of the model specification, the validity of the instruments, and the theoretical underpinnings of the claim is essential before drawing a firm conclusion.

### Part2

In [30]:
df2 = pd.read_csv('/Users/youziya/OneDrive - York University/MBAN 5110 Predictive Modelling/Midterm/midterm_parttwo.csv')

In [31]:
df2.head()

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative


In [ ]:
# One-hot encoding the 'Credit Rating' column
credit_rating_encoded = pd.get_dummies(df2['Credit Rating'], drop_first=True)
data_parttwo_encoded = pd.concat([df2.drop('Credit Rating', axis=1), credit_rating_encoded], axis=1)

# Assuming the rest of the data preprocessing (encoding other categorical variables) is done

# Defining X (features) and y (target)
X = pd.concat([data_parttwo.drop(categorical_cols + ['Credit Rating'], axis=1), encoded_categorical_df], axis=1)

# Splitting the dataset into training (50%) and test (50%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Rest of the logistic regression code follows...


In [ ]:
data_parttwo_encoded['Credit Rating'] = data_parttwo_encoded['Credit Rating'].map({'Positive': 1, 'Negative': 0})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
log_reg.fit(X_train, y_train)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split

# Splitting the dataset into training (50%) and test (50%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Fitting a logistic regression model with class weights adjusted for imbalance
log_reg_balanced = LogisticRegression(max_iter=1000, class_weight='balanced')
log_reg_balanced.fit(X_train, y_train)

# Predicting on the test set
y_pred_balanced = log_reg_balanced.predict(X_test)

# Confusion Matrix, Recall, Precision, and F1 Score for the adjusted model
conf_matrix_balanced = confusion_matrix(y_test, y_pred_balanced)
recall_balanced = recall_score(y_test, y_pred_balanced)
precision_balanced = precision_score(y_test, y_pred_balanced)
f1_balanced = f1_score(y_test, y_pred_balanced)

# Display the results
print("Confusion Matrix:", conf_matrix_balanced)
print("Recall:", recall_balanced)
print("Precision:", precision_balanced)
print("F1 Score:", f1_balanced)


In [ ]:

# One-hot encode the categorical variables
encoder = OneHotEncoder(sparse=False, drop='first')  # drop='first' to avoid dummy variable trap
categorical_columns = ['Requested Credit Amount', 'Number of Dependents', 'Monthly Income', 'Monthly Expense', 'Marital Status']
X_categorical = encoder.fit_transform(df2[categorical_columns])

# Combine the one-hot encoded columns with the continuous ones
X_continuous = df2[['Years of Education after High School']].values
X = np.hstack((X_continuous, X_categorical))

# Convert the target variable 'Credit Rating' into binary (0, 1)
Y = (df2['Credit Rating'] == 'Positive').astype(int).values

# Split the data into training (50%) and test (50%) sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=0)

# Fit the logistic regression model on the training set
log_reg = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence
log_reg.fit(X_train, Y_train)

# Predict the Credit Rating on the test set
Y_pred = log_reg.predict(X_test)

# Calculate the confusion matrix, recall, precision, and F1 score
conf_matrix = confusion_matrix(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

# Get the predicted probabilities for the positive class (1)
Y_probs = log_reg.predict_proba(X_test)[:, 1]

# Find the threshold that will result in 15% of applications being approved
threshold_15_percent = np.percentile(Y_probs, 85)  # 100% - 15% = 85%

# Apply the new threshold to determine credit approvals
Y_pred_adjusted = (Y_probs >= threshold_15_percent).astype(int)

# Calculate the new confusion matrix, recall, precision, and F1 score with the adjusted threshold
conf_matrix_adjusted = confusion_matrix(Y_test, Y_pred_adjusted)
recall_adjusted = recall_score(Y_test, Y_pred_adjusted)
precision_adjusted = precision_score(Y_test, Y_pred_adjusted)
f1_adjusted = f1_score(Y_test, Y_pred_adjusted)

# Output the new evaluation metrics with the adjusted threshold
(conf_matrix_adjusted, recall_adjusted, precision_adjusted, f1_adjusted, threshold_15_percent)


### New Bank Metric

In [ ]:

# One-hot encode the categorical variables
encoder = OneHotEncoder(sparse=False, drop='first')  # Avoid dummy variable trap
categorical_columns = ['Requested Credit Amount', 'Number of Dependents', 
                       'Monthly Income', 'Monthly Expense', 'Marital Status']
X_categorical = encoder.fit_transform(df2[categorical_columns])

# Combine the one-hot encoded columns with the continuous ones
X_continuous = df2[['Years of Education after High School']].values
X = np.hstack((X_continuous, X_categorical))

# Convert the target variable 'Credit Rating' to binary (0, 1)
Y = (df2['Credit Rating'] == 'Positive').astype(int).values

# Split the data into training (50%) and test (50%) sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=0)

# Fit the logistic regression model on the training set
log_reg = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence
log_reg.fit(X_train, Y_train)

# Predict the Credit Rating on the test set using the default threshold
Y_pred = log_reg.predict(X_test)

# Calculate the confusion matrix, recall, precision, and F1 score
conf_matrix = confusion_matrix(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

# Get the predicted probabilities for the positive class (1)
Y_probs = log_reg.predict_proba(X_test)[:, 1]

# Find the threshold that results in 15% of applications being approved
threshold_15_percent = np.percentile(Y_probs, 85)  # 100% - 15% = 85%

# Apply the new threshold to determine credit approvals
Y_pred_adjusted = (Y_probs >= threshold_15_percent).astype(int)

# Calculate the new confusion matrix, recall, precision, and F1 score with the adjusted threshold
conf_matrix_adjusted = confusion_matrix(Y_test, Y_pred_adjusted)
recall_adjusted = recall_score(Y_test, Y_pred_adjusted)
precision_adjusted = precision_score(Y_test, Y_pred_adjusted)
f1_adjusted = f1_score(Y_test, Y_pred_adjusted)

# Output the new evaluation metrics with the adjusted threshold
original_metrics = {
    "Confusion Matrix": conf_matrix.tolist(),
    "Recall": recall,
    "Precision": precision,
    "F1 Score": f1
}

adjusted_metrics = {
    "Confusion Matrix": conf_matrix_adjusted.tolist(),
    "Recall": recall_adjusted,
    "Precision": precision_adjusted,
    "F1 Score": f1_adjusted,
    "Threshold": threshold_15_percent
}

original_metrics, adjusted_metrics
